In [1]:
# read from files

# cites graph
cites_rowptr = [0, 1, 4, 5, 7, 8, 8]

# cited by graph
citedby_rowptr = [0, 0, 4, 5, 6, 7, 8]
citedby_colind = [0, 2, 3, 4, 1, 1, 1, 3]

In [2]:
# construct graph in a readable manner

graph = []
for i in range(len(citedby_rowptr) - 1):
    graph.append([])
for i in range(len(citedby_rowptr) - 1):
    for j in range(citedby_rowptr[i], citedby_rowptr[i + 1]):
        graph[citedby_colind[j]].append(i)

In [3]:
# construct cites many list
cites_many = []
for i in range(len(cites_rowptr) - 1):
    cites_many.append(cites_rowptr[i + 1] - cites_rowptr[i] + 1)
cites_many.append(len(cites_rowptr) - 1)

In [4]:
# calculate pagerank

num_vertices = len(citedby_rowptr)

pagerank = [1 / num_vertices for i in range(num_vertices)]

step = 0
error = 1
while error > 0.0001:
    step += 1
    error = 0
    new_pagerank = [x for x in pagerank]
    
    # for original nodes
    for i in range(num_vertices - 1):
        curr_rank = 0.15 / num_vertices
        for j in range(citedby_rowptr[i], citedby_rowptr[i + 1]):
            curr_rank += 0.85 * pagerank[citedby_colind[j]] / cites_many[citedby_colind[j]]
        curr_rank += 0.85 * pagerank[-1] / (num_vertices - 1)
        new_pagerank[i] = curr_rank
        error += abs(new_pagerank[i] - pagerank[i]) / num_vertices

    # for additional node
    curr_rank = 0.15 / num_vertices
    for j in range(num_vertices - 1):
        curr_rank += 0.85 * pagerank[j] / cites_many[j]
    new_pagerank[-1] = curr_rank
    error += abs(new_pagerank[-1] - pagerank[-1]) / num_vertices
     
    pagerank = [x for x in new_pagerank]    
    print("Step: {:2d}    Error: {:.12f}".format(step, error))

Step:  1    Error: 0.106972789116
Step:  2    Error: 0.067580782313
Step:  3    Error: 0.041109935988
Step:  4    Error: 0.024610838884
Step:  5    Error: 0.014809333232
Step:  6    Error: 0.008790610091
Step:  7    Error: 0.005167738358
Step:  8    Error: 0.003017321072
Step:  9    Error: 0.001753164350
Step: 10    Error: 0.001015069847
Step: 11    Error: 0.000586217699
Step: 12    Error: 0.000337917642
Step: 13    Error: 0.000194521718
Step: 14    Error: 0.000111863402
Step: 15    Error: 0.000064281459


In [5]:
# print pagerank to a file
with open('pagerank.txt', 'w') as out:
    out.write(str(len(pagerank)) + "\n\n")
    for val in pagerank:
        out.write("{:.12f}\n".format(val))    

In [6]:
# read authorwritespaper graph

awp_rowptr = []
with open('authorwritespaper_rowptr.txt') as inp:
    inp.readline()
    inp.readline()
    for line in inp:
        awp_rowptr.append(int(line[:-1]))
        
awp_colind = []
with open('authorwritespaper_colind.txt') as inp:
    inp.readline()
    inp.readline()
    for line in inp:
        awp_colind.append(int(line[:-1]))

In [7]:
# compute impact list

impact = []
for i in range(len(citedby_rowptr) - 1):
    total = 0
    for j in range(citedby_rowptr[i], citedby_rowptr[i + 1]):
        total += pagerank[citedby_colind[j]]
    impact.append(total)

In [8]:
# compute hprimes

hprime = []
for i in range(len(awp_rowptr) - 1):
    papers = []
    for j in range(awp_rowptr[i], awp_rowptr[i + 1]):
        papers.append((awp_colind[j], impact[awp_colind[j]]))
    sorted_papers = [key for key, val in sorted(papers, key=lambda x: x[1], reverse=True)]
    
    h = 0
    while h < len(sorted_papers):
        lhs = impact[sorted_papers[h]]
        rhs = h * (1 - pagerank[-1]) /(len(pagerank) - 1) 
        
        if lhs >= rhs: h += 1
        else: break
    hprime.append(h)

In [9]:
# print hprimes to a file
with open('hprime.txt', 'w') as out:
    out.write(str(len(hprime)) + "\n\n")
    for val in hprime:
        out.write("{}\n".format(val))   